# SariCoach: AI Retail Intelligence Demo

This notebook demonstrates the core agentic logic of SariCoach. In the production application, this logic is encapsulated within the `saricoach` Python package and exposed via a FastAPI backend to a React mobile dashboard.

## Architecture

1.  **Data Layer**: Supabase (PostgreSQL) for transactional data, with a CSV fallback for Kaggle environments.
2.  **Agent Layer**: `saricoach` package containing:
    *   `PlannerAgent`: Orchestrates the analysis workflow.
    *   `DataAnalystAgent`: Queries data and calculates KPIs.
    *   `CoachAgent`: Generates actionable advice using LLMs (Gemini) or heuristics.
3.  **Service Layer**: FastAPI (`service/`) providing REST endpoints.
4.  **Presentation Layer**: React Mobile Dashboard (`src/`).

## Setup

First, we load the necessary modules and initialize the context.

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
sys.path.append(str(project_root))

from saricoach.data_context import DataContext
from saricoach.backends.csv_backend import build_context_from_csv
from saricoach.agents.planner import PlannerAgent
from saricoach.agents.data_analyst import DataAnalystAgent
from saricoach.agents.coach import CoachAgent

# Initialize Data Context (CSV Mode for Demo)
data_dir = project_root / "data" / "processed"
ctx = build_context_from_csv(data_dir)
print("Data Context Loaded")

## Scenario 1: Daily Store Check

The store owner opens the app. The `PlannerAgent` triggers a routine analysis.

In [ ]:
# Initialize Agents
planner = PlannerAgent()
analyst = DataAnalystAgent(ctx=ctx)
coach = CoachAgent(model_name="heuristic") # Use 'gemini' if API key is set

# 1. Planner decides what to do
plan = planner.create_plan(goal="daily_store_check")
print(f"Plan: {plan}")

# 2. Analyst executes data retrieval
summary = analyst.analyze_store_performance(store_id=1)
print("Store Summary:", summary)

## Scenario 2: AI Coaching

Based on the analysis, the `CoachAgent` provides a recommendation.

In [ ]:
# 3. Coach generates advice
advice = coach.generate_advice(summary)
print(f"Coach Says: {advice}")